In [4]:
import os
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

class StudentScore(object):
    def __init__(self, className: str, studentName: str , regularScore: float, middleScore: float ,finalScore: float):
        self.className=className
        self.studentName=studentName
        self.regularScore=regularScore
        self.middleScore=middleScore
        self.finalScore=finalScore
        

class EduSys:
    url='https://zhxy.jxyjxy.com:8078/lyuapServer/login?service=http://218.87.96.236:4106/shiro-cas'
    user_xpath='//*[@id="userName"]'
    psw_xpath='//*[@id="password"]'
    login_xpath='/html/body/div[1]/span/div[4]/div[2]/div/div[1]/div/div/form/div[5]/button[1]'
    auth_xpath='//*[@id="captcha"]'
    JWXT_xpath='//*[@id="serviceContainer46287ad0"]/div/div/div/div/div/div[2]/div/div/div/div[2]'
    def start_Edge(self):
        self.driver=webdriver.Edge()        
        self.driver.implicitly_wait(10)# seconds,等待时间,如果网页元素节点没加载,默认等待10s。
        self.driver.get(url=EduSys.url)
        assert "统一身份认证平台" in self.driver.title

        #找到用户名输入栏
        user_name=self.driver.find_element(By.XPATH,self.user_xpath)
        user_name.send_keys("202109010")
        #找到密码输入栏
        psw=self.driver.find_element(By.XPATH,self.psw_xpath)
        psw.send_keys("pxa@961224")
        #找到验证码输入栏
        authCode=self.driver.find_element(By.XPATH,self.auth_xpath)
        user_input=input('请输入验证码:\n')
        authCode.send_keys(user_input)
        authCode.send_keys(Keys.ENTER)
        self.driver.find_element(By.XPATH,'//*[@id="root"]/div/div/div[4]/div[1]/div/div/div/div[1]/div[2]/div/div/div/div[1]/div/div/div[1]/div/div/div[2]/div[1]/div/div/div/div/div[1]/div[2]').click()
        self.driver.find_element(By.NAME,"教务系统").click()
        self.driver.find_element(By.LINK_TEXT,'学生成绩录入').click()
        
        


    def read_excel(self,path):
        """"读取excel成绩数据"""
        df=pd.read_excel(path)
        StudentScore.studentName=df['学生姓名']
        StudentScore.className=df['班级']
        StudentScore.regularScore=df['指导']
        StudentScore.middleScore=df['评阅']
        StudentScore.finalScore=df['答辩']

    def run(self):
        try:
            path='毕业论文成绩汇总表.xlsx'
            if os.path.exists(path):
                self.read_excel(path)
                self.start_Edge()
            else:
                print('文件不存在')
        except Exception as e:
            import traceback
            print(traceback.print_exc())

if __name__=='__main__':
    Edu_sys=EduSys()
    Edu_sys.run()

None


Traceback (most recent call last):
  File "C:\Users\13294\AppData\Local\Temp\ipykernel_45244\2667040570.py", line 62, in run
    self.start_Edge()
  File "C:\Users\13294\AppData\Local\Temp\ipykernel_45244\2667040570.py", line 43, in start_Edge
    self.driver.find_element(By.LINK_TEXT,'学生成绩录入').click()
  File "C:\Users\13294\AppData\Roaming\Python\Python310\site-packages\selenium\webdriver\remote\webdriver.py", line 742, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
  File "C:\Users\13294\AppData\Roaming\Python\Python310\site-packages\selenium\webdriver\remote\webdriver.py", line 348, in execute
    self.error_handler.check_response(response)
  File "C:\Users\13294\AppData\Roaming\Python\Python310\site-packages\selenium\webdriver\remote\errorhandler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate el

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pytesseract
import sympy as sp
from PIL import Image, ImageOps, ImageFilter,ImageEnhance
import string
import requests
from io import BytesIO
import base64

# 调用 EDGE 浏览器
driver = webdriver.Edge()
url = "https://zhxy.jxyjxy.com:8078/lyuapServer/login?service=http://218.87.96.236:4106/shiro-cas"
driver.get(url)

# 等待页面加载完成
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'form')))

# 使用WebDriver的page_source属性获取页面源代码
page_source = driver.page_source

# 使用BeautifulSoup解析页面源代码
soup = BeautifulSoup(page_source, 'html.parser')

# 直接查找验证码图片元素
img = driver.find_element(By.CSS_SELECTOR, 'form img')
img_src = img.get_attribute('src')
print("Image source:", img_src)


def preprocess_image(image):
    # 调整大小
    image = image.resize((300, 100))
    # 灰度化
    image = ImageOps.grayscale(image)
    # 二值化
    threshold = 150  # 调整阈值
    image = image.point(lambda p: p > threshold and 255)

    # 去噪
    image = image.filter(ImageFilter.MedianFilter(size=3))

    # 增强对比度
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)

    # 显示预处理后的图像（可选）
    #image.show()

    return image

# Set the path to the Tesseract executable (update this path according to your installation)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def recognize_and_calculate_result(img_src):
    
    base64_image=img_src.replace("data:image/png;base64,","")
    # 将图片数据读取为二进制流
    img_data =  base64.b64decode(base64_image)
    with open("temp_image.png", "wb") as img_file:
        img_file.write(img_data)
    # Open the image
    image = Image.open("temp_image.png")
    image.show()

    # Apply preprocessing (you can customize this based on your needs)
    image = preprocess_image(image)

    # Use Tesseract OCR to extract text from the image
    recognize_text = pytesseract.image_to_string(image, config='--psm 6 -c tessedit_char_whitelist=0123456789+/*=?')
    expression_text = recognize_text.replace("=?","")

    try:
        # Evaluate the expression using sympy
        result = sp.sympify(expression_text)
        return expression_text, result
    except sp.SympifyError:
        return expression_text, "Error: Unable to evaluate the expression"

if __name__ == "__main__":
    # Provide the path to the image containing the math expression

    expression, result = recognize_and_calculate_result(img_src)

    print("Math Expression:", expression)
    print("Result:", result)

In [ ]:
from PIL import Image
import base64
from io import BytesIO

# 图像数据
image_data = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACY0lEQVR42r2ZPU4EMQyFcxUKGkTJ%0AAfYONBQcgCtQAFei4QK0nICOozBSpGD55/nZM4NkrWZnM9nky7PjeMbP91dqH5/vy9ZX0Djtjfw7%0AZmzRI3LMtqtq/9NGFVY6AtAADFTe3AMr4rV/PVhYcjIlWFEzUrzVyaTrJAeQtrS/jp7OS7CiBWdg%0A9VRGDj513iYs1XU0AUZcjDCZuMOHIZcIGezkT6MnckkNOCkWF+NH58HCIfIsWOA+GBnjyLg9c4ds%0AyYzkMFjr4uZyNQ3Dunu8TyPaZpfrF2XpnElxPb89LMPqOxKW4qVgqcaLkb2wfm1JWV49WJLUMkZc%0Ao+f8LqzJaMKKPHHSWZ9g2hsX9Wukr2oonGqSX5e+mrDIzU45oLyIts6N0TQ+WivHZGBFgc/2ScLa%0AbPSWyF4vNVllKWNgRVoDsG6fXrFFYd6F5W5cA2wWaYq0LqaUFCzXJiP5iWG5McsVCAlLzSiSlZsG%0ADX4viESntj8Aa9KR0SrlFcHCiRi5/FFoZ2GlZzR7RzkdVpaCVTozROJi6iIRqWjXcp9CsECiIB0Q%0A2P4KjMvLfZyPWSpUMan8HyyQNKdaa8AqlZPUgu2HBTbBg2Exh5XUDUlYM89yE3qyDmNPr2kJCPc8%0AmOO+/Ne04mFhNWqVIIMnHTmtF7nRPYdVrTT+A6x0K+TrEJHHLFhprQ3BSnkBSR9Y8GYEUhIX9qEC%0ALMAFZ2GlomA70vdguUO1BZmysiIuTI5H1kVLlaZ2gb9XlS+7IVM2ZIbOv2JhqpTqaJW+NGFqeO1t%0AekTndabmT75faLwZY45cZ4gLN/sF5NbzaOYQqkgAAAAASUVORK5CYII="
#去除换行符
image_data.strip()
# 移除数据头部
base64_data = image_data.replace("data:image/png;base64,", "")

# 添加正确的填充
missing_padding = len(base64_data) % 4
if missing_padding:
    base64_data += '=' * (4 - missing_padding)

# 解码 base64 数据
image_bytes = base64.b64decode(base64_data)

# 读取为图像对象
image = Image.open(BytesIO(image_bytes))

# 显示图像（可选）
image.show()


In [ ]:

import base64
import numpy as np
import cv2

def base64img2file(imgname:str,imgsrc: str):
    img_bytes=imgsrc.split(',')[1]
    
    #将base64转码为byte，然后再使用opencv转image对象
    img_data = base64.b64decode(img_bytes)
    nparr = np.fromstring(img_data, np.uint8)
    img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    if img_np is not None:
        cv2.imwrite(imgname,img_np)

base64str="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACY0lEQVR42r2ZPU4EMQyFcxUKGkTJ%0AAfYONBQcgCtQAFei4QK0nICOozBSpGD55/nZM4NkrWZnM9nky7PjeMbP91dqH5/vy9ZX0Djtjfw7%0AZmzRI3LMtqtq/9NGFVY6AtAADFTe3AMr4rV/PVhYcjIlWFEzUrzVyaTrJAeQtrS/jp7OS7CiBWdg%0A9VRGDj513iYs1XU0AUZcjDCZuMOHIZcIGezkT6MnckkNOCkWF+NH58HCIfIsWOA+GBnjyLg9c4ds%0AyYzkMFjr4uZyNQ3Dunu8TyPaZpfrF2XpnElxPb89LMPqOxKW4qVgqcaLkb2wfm1JWV49WJLUMkZc%0Ao+f8LqzJaMKKPHHSWZ9g2hsX9Wukr2oonGqSX5e+mrDIzU45oLyIts6N0TQ+WivHZGBFgc/2ScLa%0AbPSWyF4vNVllKWNgRVoDsG6fXrFFYd6F5W5cA2wWaYq0LqaUFCzXJiP5iWG5McsVCAlLzSiSlZsG%0ADX4viESntj8Aa9KR0SrlFcHCiRi5/FFoZ2GlZzR7RzkdVpaCVTozROJi6iIRqWjXcp9CsECiIB0Q%0A2P4KjMvLfZyPWSpUMan8HyyQNKdaa8AqlZPUgu2HBTbBg2Exh5XUDUlYM89yE3qyDmNPr2kJCPc8%0AmOO+/Ne04mFhNWqVIIMnHTmtF7nRPYdVrTT+A6x0K+TrEJHHLFhprQ3BSnkBSR9Y8GYEUhIX9qEC%0ALMAFZ2GlomA70vdguUO1BZmysiIuTI5H1kVLlaZ2gb9XlS+7IVM2ZIbOv2JhqpTqaJW+NGFqeO1t%0AekTndabmT75faLwZY45cZ4gLN/sF5NbzaOYQqkgAAAAASUVORK5CYII===="
imgname='image/%s.png'
base64img2file(imgname,base64str)
